<a href="https://colab.research.google.com/github/jankovicsandras/ml/blob/main/phi2_all_MiniLM_L6_v2_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Check if model file is downloaded

In [9]:
! ls -la
! pwd

total 2024124
drwxr-xr-x 1 root root       4096 Jan 22 18:41 .
drwxr-xr-x 1 root root       4096 Jan 22 18:40 ..
drwxr-xr-x 4 root root       4096 Jan 19 14:19 .config
-rw-r--r-- 1 root root 2072682336 Dec 18 20:25 phi-2.Q5_K_M.gguf
drwxr-xr-x 1 root root       4096 Jan 19 14:20 sample_data
/content


# Model download : Do this only if model file is not visibile above!

In [10]:
! wget https://huggingface.co/TheBloke/phi-2-GGUF/resolve/main/phi-2.Q5_K_M.gguf

--2024-01-22 18:47:35--  https://huggingface.co/TheBloke/phi-2-GGUF/resolve/main/phi-2.Q5_K_M.gguf
Resolving huggingface.co (huggingface.co)... 52.85.151.31, 52.85.151.16, 52.85.151.66, ...
Connecting to huggingface.co (huggingface.co)|52.85.151.31|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/df/73/df7366134ed3798e8afca879a1fb9eb2741134a7d549d744d7163b5d84cf3e2d/ce6bcc8fa75b467e1eb693bcc921fe366ceac14aa92ec3dc84da536c2289510a?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27phi-2.Q5_K_M.gguf%3B+filename%3D%22phi-2.Q5_K_M.gguf%22%3B&Expires=1706208455&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwNjIwODQ1NX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2RmLzczL2RmNzM2NjEzNGVkMzc5OGU4YWZjYTg3OWExZmI5ZWIyNzQxMTM0YTdkNTQ5ZDc0NGQ3MTYzYjVkODRjZjNlMmQvY2U2YmNjOGZhNzViNDY3ZTFlYjY5M2JjYzkyMWZlMzY2Y2VhYzE0YWE5MmVjM2RjODRkYTUzNmMyMjg

# Install deps


In [11]:
!pip -q install langchain llama-cpp-python sentence-transformers langchain-community faiss-cpu

# naive vectordb class

In [12]:
# naive vectordb
class Nvdb :
  def __init__(self,embedder) :
    self.embedder = embedder
    self.docs = []
    self.vecs = []

  def insert(self,doc) :
    self.docs.append(doc)
    self.vecs.append(self.embedder.encode(doc.page_content))

  def insertarr(self,docarr) :
    for d in docarr : self.insert(d)

  def selecttopk(self,k,query) :
    v = self.embedder.encode([query])
    sc = [ [self.similarityfunction(v,x),i] for i,x in enumerate(self.vecs) ]
    sc.sort(key=lambda x : x[0], reverse=True)
    return [ [x[0],x[1],self.docs[x[1]]] for x in sc[:k]]

  def similarityfunction(self,v1,v2) :
    x = np.dot(v1,v2)
    return x


# test shop items

In [13]:
import random

colors = ['black','blue','green','cyan','red','magenta','brown','light grey','dark grey','bright blue']
sizes = [str(30+x*2) for x in range(0,10)]
itemtypes = [ 'belt','cap','hat','jeans','jumper', 'shirt','shorts','sneakers','suit','tie' ]
adjs = ['Fantastic','Cool','Superb','Awesome','Trendy']
allitems = []
for c in colors :
  for s in sizes :
    for ii,i in enumerate(itemtypes) :
      allitems.append( random.choice(adjs)+' '+c+' '+i+' in size '+s+'. Price: '+str(int(s)+20+5*ii)+' USD.' )

#print(allitems)


# naive and FAISS vectordbs

In [14]:
from sentence_transformers import SentenceTransformer
from langchain.schema.document import Document
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
import numpy as np

# Test shop db
itemnum = 20
shopdb = [ Document(page_content=random.choice(allitems), metadata={"source": "local","itemid":i}) for i in range(0,itemnum)]
print(str(shopdb).replace('),','),\n'))

# naive vectordb init with HuggingFace all-MiniLM-L6-v2 embedder
embeddermodel = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
myNvdb = Nvdb(embeddermodel)
myNvdb.insertarr(shopdb)

# FAISS vectordb init with LangChain/HuggingFace all-MiniLM-L6-v2 embedder
langchainembeddermodel = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
faissdb = FAISS.from_documents(shopdb, langchainembeddermodel)

# test questions
questions = [
    'I want to buy a hat. What colors do you have?',
    'Can you recommend something green?',
    'Do you have shirts under 50 USD?',
    'What do you have in size 40?',
    'I would like to buy sneakers for my friend. Do you have something in size 46, preferably cyan or blue?',
    'What can you recommend in red?'
]

mytopk = 3

# Retrieval comparison
for q in questions:
  print('---------------------------------')
  print(q)
  r1 = myNvdb.selecttopk(mytopk,q)
  print('myNvdb')
  #print(str(r1).replace('),','),\n'))
  for rr in r1 :
    print(rr[2].page_content,'|',rr[0][0])

  r2 = faissdb.similarity_search_with_score(q)[:mytopk]
  print('FAISS')
  #print(str(r2).replace('),','),\n'))
  for rr in enumerate(r2) :
    print(rr[1][0].page_content,'|',rr[1][1])



[Document(page_content='Superb blue belt in size 36. Price: 56 USD.', metadata={'source': 'local', 'itemid': 0}),
 Document(page_content='Cool dark grey suit in size 30. Price: 90 USD.', metadata={'source': 'local', 'itemid': 1}),
 Document(page_content='Fantastic cyan jeans in size 40. Price: 75 USD.', metadata={'source': 'local', 'itemid': 2}),
 Document(page_content='Fantastic cyan jeans in size 40. Price: 75 USD.', metadata={'source': 'local', 'itemid': 3}),
 Document(page_content='Cool cyan sneakers in size 46. Price: 101 USD.', metadata={'source': 'local', 'itemid': 4}),
 Document(page_content='Cool cyan cap in size 48. Price: 73 USD.', metadata={'source': 'local', 'itemid': 5}),
 Document(page_content='Awesome light grey jeans in size 44. Price: 79 USD.', metadata={'source': 'local', 'itemid': 6}),
 Document(page_content='Cool cyan cap in size 48. Price: 73 USD.', metadata={'source': 'local', 'itemid': 7}),
 Document(page_content='Cool green jumper in size 48. Price: 88 USD.', m

# LLMChain

In [15]:
from langchain import PromptTemplate, LLMChain, LlamaCpp
from langchain.callbacks import StreamingStdOutCallbackHandler

# Load model
model_name = 'phi-2.Q5_K_M.gguf'
llm = LlamaCpp(
	model_path=model_name,
#  n_ctx=2048,
#  seed=0,
  temperature=0.1,
#  max_tokens=128,
  verbose=True,
  callbacks=[StreamingStdOutCallbackHandler()],
#  use_mmap=True, use_mlock=True,
#  stop=['\nObservasjon','\n\n\n\n'],
)

# Template and Prompt
template = """Answer the following question:\n{question}\nUse the following information:\n{ragresult}"""

prompt = PromptTemplate( template=template, input_variables=['question','ragresult'], verbose=True )

# LLMChain
llm_chain = LLMChain(prompt=prompt, llm=llm, verbose=True)


AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 
Model metadata: {'tokenizer.ggml.unknown_token_id': '50256', 'tokenizer.ggml.eos_token_id': '50256', 'tokenizer.ggml.bos_token_id': '50256', 'general.architecture': 'phi2', 'general.name': 'Phi2', 'phi2.context_length': '2048', 'general.quantization_version': '2', 'tokenizer.ggml.model': 'gpt2', 'tokenizer.ggml.add_bos_token': 'false', 'phi2.embedding_length': '2560', 'phi2.attention.head_count': '32', 'phi2.attention.head_count_kv': '32', 'phi2.feed_forward_length': '10240', 'phi2.attention.layer_norm_epsilon': '0.000010', 'phi2.block_count': '32', 'phi2.rope.dimension_count': '32', 'general.file_type': '17'}


# LangChain LLMChain question and run

In [16]:

for q in questions :
  print('-------------------------------------')

  r1 = myNvdb.selecttopk(mytopk,q)
  rres = ''
  for rr in r1 :
    rres += rr[2].page_content + '\n'

  result = llm_chain.invoke( { 'question':q, 'ragresult':rres } )
  print(result['text'])


-------------------------------------


> Entering new LLMChain chain...
Prompt after formatting:
Answer the following question:
I want to buy a hat. What colors do you have?
Use the following information:
Trendy light grey cap in size 34. Price: 59 USD.
Trendy light grey cap in size 34. Price: 59 USD.
Cool dark grey suit in size 30. Price: 90 USD.


> Finished chain.

-------------------------------------


> Entering new LLMChain chain...
Prompt after formatting:
Answer the following question:
Can you recommend something green?
Use the following information:
Superb green sneakers in size 40. Price: 95 USD.
Cool green jumper in size 48. Price: 88 USD.
Cool dark grey suit in size 30. Price: 90 USD.



Llama.generate: prefix-match hit


A: The best option would be the Superb green sneakers in size 40, as it is the most affordable and comes in a color that is suitable for many occasions. It is also a versatile pair of shoes that can be dressed up or down depending on the occasion.

> Finished chain.
A: The best option would be the Superb green sneakers in size 40, as it is the most affordable and comes in a color that is suitable for many occasions. It is also a versatile pair of shoes that can be dressed up or down depending on the occasion.

-------------------------------------


> Entering new LLMChain chain...
Prompt after formatting:
Answer the following question:
Do you have shirts under 50 USD?
Use the following information:
Cool cyan shirt in size 44. Price: 89 USD.
Superb red shirt in size 42. Price: 87 USD.
Cool dark grey suit in size 30. Price: 90 USD.



Llama.generate: prefix-match hit


A: No, we do not have shirts under 50 USD.

> Finished chain.
A: No, we do not have shirts under 50 USD.

-------------------------------------


> Entering new LLMChain chain...
Prompt after formatting:
Answer the following question:
What do you have in size 40?
Use the following information:
Superb green sneakers in size 40. Price: 95 USD.
Fantastic blue sneakers in size 48. Price: 103 USD.
Fantastic cyan jeans in size 40. Price: 75 USD.



...

Answer: The fantastic blue sneakers are in size 48 and cost $103.

> Finished chain.
Answer: The fantastic blue sneakers are in size 48 and cost $103.

-------------------------------------


> Entering new LLMChain chain...
Prompt after formatting:
Answer the following question:
I would like to buy sneakers for my friend. Do you have something in size 46, preferably cyan or blue?
Use the following information:
Cool cyan sneakers in size 46. Price: 101 USD.
Fantastic blue sneakers in size 48. Price: 103 USD.
Superb green sneakers in size 40. Price: 95 USD.



Llama.generate: prefix-match hit


Excellent yellow sneakers in size 42. Price: 99 USD.
A: Yes, we have the cool cyan sneakers in size 46 for you! They are priced at 101 USD. Would you like to purchase them?

> Finished chain.
Excellent yellow sneakers in size 42. Price: 99 USD.
A: Yes, we have the cool cyan sneakers in size 46 for you! They are priced at 101 USD. Would you like to purchase them?

-------------------------------------


> Entering new LLMChain chain...
Prompt after formatting:
Answer the following question:
What can you recommend in red?
Use the following information:
Superb red shirt in size 42. Price: 87 USD.
Cool dark grey suit in size 30. Price: 90 USD.
Fantastic blue sneakers in size 48. Price: 103 USD.



Llama.generate: prefix-match hit


A: The best recommendation would be to purchase the Superb red shirt in size 42 for 87 USD as it is a great quality item at an affordable price.

> Finished chain.
A: The best recommendation would be to purchase the Superb red shirt in size 42 for 87 USD as it is a great quality item at an affordable price.

